In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from surp import subgiants
import surp
import vice
from surp import gce_math as gcem
import arya

import sys

In [ ]:
import warnings;
warnings.filterwarnings("ignore")

In [ ]:
from singlezone import run_singlezone, exp_sfh

In [ ]:
y_scale = 0.6# 0.712

mw = {}
gse = dict(eta=y_scale * 9.56, t_end=10.73, tau_star=26.60, tau_sfh=2.18, sfh=exp_sfh(None), mode="ifr")
wukong = dict(eta=y_scale * 48, t_end=3.4, tau_star=45, tau_sfh=3.08, sfh=exp_sfh(None), mode="ifr")

In [ ]:
def calc_model(**params):
    sz, h = run_singlezone(**params)
    
    return h.MG_H, h.C_MG, h.sfr

In [ ]:
def plot_sz(label="", **kwargs,):
    x, y, w = calc_model(**kwargs)
    plt.scatter(x, y, s=w, label=label)
    plt.xlabel("[Mg/Fe]")
    plt.ylabel("[C/Mg]")
    arya.Legend(loc=-1, color_only=True)

In [ ]:
amarsi19 = pd.read_csv(surp.DATA_DIR + "stellar_catalogues/amarsi_19.tsv", delimiter="\t *", comment="#", skiprows=[62, 63],)

amarsi19["FE_H"] = amarsi19["[Fe/H]3L"]
amarsi19["FE_H_err"] = amarsi19["e_[Fe/H]3L"]

amarsi19["O_H"] = amarsi19["[O/H]3N"]
amarsi19["O_H_err"] = amarsi19["e_[O/H]3N"]

amarsi19["C_H"] = amarsi19["[C/H]3N"]
amarsi19["C_H_err"] = amarsi19["e_[C/H]3N"]

amarsi19["C_O"] = amarsi19["C_H"] - amarsi19["O_H"] 
amarsi19["C_O_err"] = amarsi19["O_H_err"] + amarsi19["C_H_err"] 

amarsi19["O_FE"] = amarsi19["O_H"] - amarsi19["[Fe/H]3L"]
amarsi19["O_FE_err"] = amarsi19["e_[Fe/H]3L"] + amarsi19["O_H_err"]

amarsi19_sun = amarsi19.iloc[151]
amarsi19.drop(index=151, inplace=True)

In [ ]:
def plot_obs(**kwargs):
    plt.scatter(amarsi19.O_H, amarsi19.C_O, zorder=-1, s=0.1, label="amarsi+19")
    

In [ ]:
def plot_all():
    plot_sz(label="solar")
    plot_sz(**gse, label="gse")
    plot_sz(**wukong, label="wukong")
    plot_obs()
    arya.Legend(color_only=True)
    plt.xlim(-4)

In [ ]:
models = "../models/"

In [ ]:
def get_y_params(name):
    return surp.YieldParams.from_file("../models/" + name + "/yield_params.toml")

In [ ]:
y_fiducial = get_y_params('fiducial')

In [ ]:
y_lin = get_y_params('fruity/cc_BiLogLin')

In [ ]:
y_fiducial_a = get_y_params('analytic/mc_best')

In [ ]:
y_lin_a = get_y_params('analytic/linear')

In [ ]:
surp.set_yields(y_fiducial, verbose=False)
plot_all()

surp.set_yields(y_fiducial_a)
plot_sz(label="solar (ana)")
plot_sz(**gse, label="gse (ana)")
plot_sz(**wukong, label="wukong (ana)")

In [ ]:
surp.set_yields(y_lin, verbose=False)
plot_all()

surp.set_yields(y_lin_a)
plot_sz(label="solar (ana)")
plot_sz(**gse, label="gse (ana)")
plot_sz(**wukong, label="wukong (ana)")

## piecewise chaos 

In [ ]:
print(vice.yields.ccsne.settings["c"])

In [ ]:
m_h_0 = (0.867 - 2.30) / 1.62

In [ ]:
gcem.MH_to_Z(m_h_0)

In [ ]:
y_piecewise = surp.yield_models.Piecewise_CC(
    [2e-3, 8.67e-4, 2.30e-3],
    [0, 0, 1.62e-3],
    [1.6e-5, 0.0021])


In [ ]:
vice.yields.ccsne.settings["c"] = y_piecewise

In [ ]:
plot_all()

In [ ]:
y_mg = vice.yields.ccsne.settings["mg"]

In [ ]:
y_spw = surp.yield_models.Piecewise_CC(
    [3*y_mg, 1*y_mg, 4*y_mg, 4*y_mg],
    [0, 0, 0, 2.5*y_mg],
    [1.6e-5, 0.006, 0.016])

In [ ]:
vice.yields.ccsne.settings["c"] = y_spw

In [ ]:
surp.set_yields(get_y_params("C11_alpha_agb1.0_m_factor0.5"), verbose=False)
vice.yields.ccsne.settings["c"] = y_spw

In [ ]:
plot_all()

In [ ]:

x = np.linspace(-5, 1, 1000)
z = gcem.MH_to_Z(x)
y = [y_piecewise(zz) for zz in z]

plt.plot(x, y)

y = [y_spw(zz) for zz in z]

plt.plot(x, y)

plt.xlabel("[M/H]")
plt.ylabel(r"$y_{\rm C}^{\rm CC}$")

In [ ]:
surp.set_yields(get_y_params("C11_f0.2_m_factor1.5"), verbose=False)
plot_all()

In [ ]:
print(vice.yields.ccsne.settings["c"])

In [ ]:
surp.set_yields(get_y_params("C11_alpha_agb1.0_m_factor0.5"), verbose=False)
plot_all()